# Financial Data Nodes / Relationships

## Libraries

In [1]:
# !pip install yahoofinancials

import json
import pandas as pd
import re
from yahoofinancials import YahooFinancials

## Load Data from Wikipedia Scraping

In [2]:
with open('new_competitors.json') as json_file:
    data = json.load(json_file)

## Create template relationship tables

In [3]:
node_type = pd.DataFrame(data=None,columns = ['Primary_Key','Node'])
node_type.head()

,Primary_Key,Node


In [4]:
edge_type = pd.DataFrame(data=None,columns = ['Edge_Primary_Key','Edge_Type'])
edge_type.head()

,Edge_Primary_Key,Edge_Type


In [19]:
edge_properties = pd.DataFrame(data=None, columns = ['Edge_Primary_Key','Edge_Type_Key','Property_Name','Property'])
edge_properties.head()

,Edge_Primary_Key,Edge_Type_Key,Property_Name,Property


In [13]:
connections = pd.DataFrame(data=None, columns = ['Node_Property_Key','Edge_Property_Key','Node_Property_Key_2'])
connections.head()

,Node_Property_Key,Edge_Property_Key,Node_Property_Key_2


In [14]:
node_properties = pd.DataFrame(data=None, columns = ['Node_Primary_Key','Node_Type_Key','Property_Name','Property'])

node_properties.head()

,Node_Primary_Key,Node_Type_Key,Property_Name,Property


In [15]:
# Preparing First Edge / relationship
edge_type = edge_type.append({'Edge_Primary_Key':1,'Edge_Type':'Has_Financials'},ignore_index=True)
edge_type.head()

,Edge_Primary_Key,Edge_Type
0,1,Has_Financials
1,1,Has_Financials


In [16]:
# Preparing First Node
node_type = node_type.append({'Primary_Key':3,'Node':'Financials'},ignore_index=True)
node_type.head()

,Primary_Key,Node
0,3,Financials
1,3,Financials


## Populate Information from Wikipedia

### Last Revenue Date

In [20]:
pattern = r'\((\d\d\d\d|\w+|FY.....)\)' # Find date last revenue reported
for i,text in enumerate(data):
    try:
        result = re.search(pattern,data[i]['Revenue']).groups()[0]
        print(data[i]['Title'], ':', result)
        # update edge properties
        edge_properties = edge_properties.append({'Edge_Primary_Key': data[i]['Title'],
                                          'Edge_Type_Key': int(4),
                                          'Property_Name': 'last_revenue_date',
                                          'Property': result},ignore_index=True)
        # update connections
        
    except:
        result = 'NaN'
        if data[i]['Title'] == 'Dell EMC':
            edge_properties = edge_properties.append({'Edge_Primary_Key': data[i]['Title'],
                                              'Edge_Type_Key': 4,
                                              'Property_Name': 'last_revenue_date',
                                              'Property': 'NaN'},ignore_index=True)
        print(data[i]['Title'], ':', result)
        continue

Cisco Systems : 2018
VMware : 2018
Big Switch Networks : NaN
D-Link : 2015
Extreme Networks : 2018
Arista Networks : 2018
Hewlett Packard Enterprise : 2018
Juniper Networks : 2018
Huawei : 2018
Netgear : 2017
Cumulus Networks : NaN
Allied Telesis : FY 2018
NEC : 2019
Mellanox Technologies : FY17
Dell EMC : NaN
ZTE : 2018
Lenovo : 2019


In [21]:
edge_properties

,Edge_Primary_Key,Edge_Type_Key,Property_Name,Property
0,Cisco Systems,4,last_revenue_date,2018
1,VMware,4,last_revenue_date,2018
2,D-Link,4,last_revenue_date,2015
3,Extreme Networks,4,last_revenue_date,2018
4,Arista Networks,4,last_revenue_date,2018
5,Hewlett Packard Enterprise,4,last_revenue_date,2018
6,Juniper Networks,4,last_revenue_date,2018
7,Huawei,4,last_revenue_date,2018
8,Netgear,4,last_revenue_date,2017
9,Allied Telesis,4,last_revenue_date,FY 2018


In [22]:
edge_properties.to_pickle('financials_edge_properties.pkl')

### Operating Income

In [12]:
pattern = r'\$(.+)(illion)' # Find operating income with "m" or "b" for millions/billions
for i,text in enumerate(data):
    try:
        result = re.search(pattern,data[i]['Operating income']).groups()[0]
        result = result.lstrip()
        if 'b' in result:
            result = round(float(result.split()[0]) * 1e9,0)
        elif ('m' in result) or ('M' in result):
            result = round(float(result.split()[0]) * 1e6,0)
        print(data[i]['Title'], ':', result)
        # update node properties
        node_properties = node_properties.append({'Node_Primary_Key': data[i]['Title'],
                                          'Node_Type_Key': 4,
                                          'Property_Name': 'operating_income',
                                          'Property': result},ignore_index=True)

    except:
        result = 'NaN'
        print(data[i]['Title'], ':', result)
        continue

Cisco Systems : 12770000000.0
VMware : 2050000000.0
Big Switch Networks : NaN
D-Link : NaN
Extreme Networks : -733000.0
Arista Networks : 273300000.0
Hewlett Packard Enterprise : 1850000000.0
Juniper Networks : 848000000.0
Huawei : 10689000000.0
Netgear : 86580000.0
Cumulus Networks : NaN
Allied Telesis : NaN
NEC : NaN
Mellanox Technologies : 118310000.0
Dell EMC : NaN
ZTE : NaN
Lenovo : 1178000.0


### Net Income

In [13]:
pattern = r'\$(.+)(illion)' # Find operating income with "m" or "b" for millions/billions
for i,text in enumerate(data):
    try:
        result = re.search(pattern,data[i]['Net income']).groups()[0]
        result = result.strip('(')
        result = result.lstrip()
        if 'b' in result:
            result = round(float(result.split()[0]) * 1e9,0)
        elif ('m' in result) or ('M' in result):
            result = round(float(result.split()[0]) * 1e6,0)        
        print(data[i]['Title'], ':', result)

        # update node properties
        node_properties = node_properties.append({'Node_Primary_Key': data[i]['Title'],
                                          'Node_Type_Key': 4,
                                          'Property_Name': 'net_income',
                                          'Property': result},ignore_index=True)
        
    except:
        result = 'NaN'
        print(data[i]['Title'], ':', result)
        continue

Cisco Systems : 110000000.0
VMware : 2420000000.0
Big Switch Networks : NaN
D-Link : NaN
Extreme Networks : -8500000.0
Arista Networks : 328100000.0
Hewlett Packard Enterprise : 1900000000.0
Juniper Networks : 306000000.0
Huawei : 8656000000.0
Netgear : 19440000.0
Cumulus Networks : NaN
Allied Telesis : 1900000.0
NEC : NaN
Mellanox Technologies : 111380000.0
Dell EMC : NaN
ZTE : NaN
Lenovo : 597000000.0


### Total Assets

In [14]:
pattern = r'\$(.+)(illion)|\(' # Find total assets
for i,text in enumerate(data):
    try:
        #print(data[i]['Total assets'])
        result = re.search(pattern,data[i]['Total assets']).groups()[0]
        result = result.lstrip()
        result = result.replace(',', '')
        if 'b' in result:
            result = round(float(result.split()[0]) * 1e9,0)
        elif 'm' in result:
            result = round(float(result.split()[0]) * 1e6,0)  
        print(data[i]['Title'], ':', result)
        # update node properties
        node_properties = node_properties.append({'Node_Primary_Key': data[i]['Title'],
                                          'Node_Type_Key': 4,
                                          'Property_Name': 'total_assets',
                                          'Property': result},ignore_index=True)
        
    except:
        result = 'NaN'
        print(data[i]['Title'], ':', result)
        continue

Cisco Systems : 108780000000.0
VMware : 14662000000.0
Big Switch Networks : NaN
D-Link : NaN
Extreme Networks : NaN
Arista Networks : 3082000000.0
Hewlett Packard Enterprise : 55490000000.0
Juniper Networks : NaN
Huawei : 97109000000.0
Netgear : 1210000000.0
Cumulus Networks : NaN
Allied Telesis : NaN
NEC : NaN
Mellanox Technologies : NaN
Dell EMC : NaN
ZTE : NaN
Lenovo : 28490000000.0


### Total Equity

In [15]:
pattern = r'\$(.+)(illion)|\(' # Find total equity
for i,text in enumerate(data):
    try:
        result = re.search(pattern,data[i]['Total equity']).groups()[0]
        result = result.lstrip()
        result = result.replace(',', '')
        if 'b' in result:
            result = round(float(result.split()[0]) * 1e9,0)
        elif 'm' in result:
            result = round(float(result.split()[0]) * 1e6,0)
        print(data[i]['Title'], ':', result)
        # update node properties
        node_properties = node_properties.append({'Node_Primary_Key': data[i]['Title'],
                                          'Node_Type_Key': 4,
                                          'Property_Name': 'total_equity',
                                          'Property': result},ignore_index=True)
        
    except:
        result = 'NaN'
        print(data[i]['Title'], ':', result)
        continue


Cisco Systems : 43200000000.0
VMware : 8097000000.0
Big Switch Networks : NaN
D-Link : NaN
Extreme Networks : NaN
Arista Networks : 2143400000.0
Hewlett Packard Enterprise : 21230000000.0
Juniper Networks : NaN
Huawei : 33994000000.0
Netgear : 730490000.0
Cumulus Networks : NaN
Allied Telesis : NaN
NEC : NaN
Mellanox Technologies : NaN
Dell EMC : NaN
ZTE : NaN
Lenovo : 51038000000.0


### Traded As & Stock Price

In [16]:
pattern = r'\:\ (\w+)' # Find stock ticker
for i,text in enumerate(data):
    try:
        #print(data[i]['Traded as'])
        result = re.search(pattern,data[i]['Traded as']).groups()[0]
        result = result.lstrip()
        result = result.replace(',', '')
        if result == 'EMC':
            result = 'DELL'
        print(data[i]['Title'], ':', result)
        # update edge properties
        # update connections
        
        # Get current stock price
        yahoo_financials = YahooFinancials(result)
        price = yahoo_financials.get_current_price()
        print('Current Price: ',price)
        
        # Get current market cap
        market_cap = yahoo_financials.get_market_cap()
        print('Current Market Cap: ',market_cap)
        
        # Get current pe
        pe = yahoo_financials.get_pe_ratio()
        print('Current P/E: ',pe)
        
        # Get current beta
        beta = yahoo_financials.get_beta()
        print('Current Beta: ',beta)
        
        # Get current dividend yield
        div_yield = yahoo_financials.get_dividend_yield()
        print('Current Div Yield: ',div_yield)
        
        
        # update node properties (stock symbol)
        node_properties = node_properties.append({'Node_Primary_Key': data[i]['Title'],
                                          'Node_Type_Key': 4,
                                          'Property_Name': 'stock_symbol',
                                          'Property': result},ignore_index=True)
        
        # update node properties (price)
        node_properties = node_properties.append({'Node_Primary_Key': data[i]['Title'],
                                          'Node_Type_Key': 4,
                                          'Property_Name': 'price',
                                          'Property': price},ignore_index=True)
        
        # update node properties (market cap)
        node_properties = node_properties.append({'Node_Primary_Key': data[i]['Title'],
                                          'Node_Type_Key': 4,
                                          'Property_Name': 'market_cap',
                                          'Property': market_cap},ignore_index=True)
        # update node properties (pe)
        node_properties = node_properties.append({'Node_Primary_Key': data[i]['Title'],
                                          'Node_Type_Key': 4,
                                          'Property_Name': 'pe_ratio',
                                          'Property': pe},ignore_index=True)
        
        # update node properties (beta)
        node_properties = node_properties.append({'Node_Primary_Key': data[i]['Title'],
                                          'Node_Type_Key': 4,
                                          'Property_Name': 'beta',
                                          'Property': beta},ignore_index=True)
        
        # update node properties (dividend yield)
        node_properties = node_properties.append({'Node_Primary_Key': data[i]['Title'],
                                          'Node_Type_Key': 4,
                                          'Property_Name': 'div_yield',
                                          'Property': div_yield},ignore_index=True)
    except:
        result = 'NaN'
        print(data[i]['Title'], ':', result)
        continue

Cisco Systems : CSCO
Current Price:  43.725
Current Market Cap:  185492815872
Current P/E:  17.39952
Current Beta:  1.204047
Current Div Yield:  0.0322
VMware : VMW
Current Price:  148.715
Current Market Cap:  60908007424
Current P/E:  9.454227
Current Beta:  1.107082
Current Div Yield:  None
Big Switch Networks : NaN
D-Link : 2332
Current Price:  None
Current Market Cap:  None
Current P/E:  None
Current Beta:  None
Current Div Yield:  None
Extreme Networks : EXTR
Current Price:  6.94
Current Market Cap:  843675008
Current P/E:  None
Current Beta:  1.663575
Current Div Yield:  None
Arista Networks : ANET
Current Price:  192.47
Current Market Cap:  14704516096
Current P/E:  20.279211
Current Beta:  1.48145
Current Div Yield:  None
Hewlett Packard Enterprise : HPE
Current Price:  15.95
Current Market Cap:  20639299584
Current P/E:  20.714287
Current Beta:  1.52739
Current Div Yield:  0.0305
Juniper Networks : JNPR
Current Price:  24.1
Current Market Cap:  8066439168
Current P/E:  22.8869

## Build Node Properties and Export

In [17]:
node_properties.to_pickle('financials_node_properties_2.pkl')

In [18]:
ticker = 'AAPL'
yahoo_financials = YahooFinancials(ticker)
yahoo_financials.get_current_price()

269.35

In [19]:
# update connections (need to create 'Financials' node first before building edges to it)
connections = connections.append({'Node_Property_Key': i+1,
                                          'Edge_Property_Key': 1,
                                          'Node_Property_Key_2': },ignore_index=True)

SyntaxError: invalid syntax (<ipython-input-19-5fd526571274>, line 4)

## Reference: Explore Scraped Data

In [ ]:
cisco_keys = list()
for key, value in cisco.items():
    print("Key: " + str(key))
    cisco_keys.append(key)

In [ ]:
arista_keys = list() 
for key, value in Arista.items():
    print("Key: " + str(key))
    arista_keys.append(key)

In [ ]:
Hewlett_Packard_keys = list()
for key, value in Hewlett_Packard.items():
    print("Key: " + str(key))
    Hewlett_Packard_keys.append(key)

In [ ]:
Juniper_keys = list()
for key, value in Juniper.items():
    print("Key: " + str(key))
    Juniper_keys.append(key)

In [ ]:
common_keys = set(cisco_keys) & set(arista_keys) & set(Hewlett_Packard_keys) & set(Juniper_keys)
print(common_keys)